## Term Frequency - Inverse Document Frequency

Aunque en este punto ya tenemos una buena representación vectorial de nuestros textos, sigue habiendo un problema: **la representación creada no está normalizada**. Esta no normalización plantea dos **problemas**:

- A nivel de *documento* (las filas de nuestra matriz de datos) **cada texto** lleva una escala completamente libre y hace **imposible compararlos** entre sí. Un **texto más largo tendrá contadores con valores mayores que un texto más corto**. En un ejemplo llevado al extremo podemos comparar un tuit con la noticia que enlaza ese tuit. Ambos documentos versarán sobre el mismo tema, pero no podrán compararse debido a la volumetría de ambos.

- A nivel de *palabras* es complicado **comparar cuáles son más relevantes y cuáles menos en un _corpus_ concreto**. Ya hemos eliminado las *palabras de parada*, pero, dependiendo del **bias** de nuestro *corpus* hay **palabras que no aportan demasiada información** y, por tanto, su incidencia en nuestro algoritmo de *machine learning* debería ser menor. Por ejemplo, imaginemos que tenemos un *corpus* de documentos que sólo hablan de los equipos de la Liga de Fútbol Profesional. En este corpus la palabra *"fútbol "* es completamente irrelevante, ya que todos los documentos hablan de ella. Por el contrario, palabras como *"lesión "* o *"fichaje "* son muy relevantes porque permiten subclasificar los documentos. Sin embargo, si nuestro *corpus* está formado por noticias de todo tipo, la palabra *'fútbol'* es muy relevante porque identifica un tipo de noticia.

Para resolver este problema, se utiliza una normalización llamada **tf-idf** (*term-frecuency times inverse document-frecuency*). Se define mediante la siguiente ecuación

$\textrm{tf-idf}(t, d) = tf(t, d) \times idf(t)$

siendo $tf(t, d)$ el número de veces que el término (palabra) $t$ aparece en el documento $d$ y $idf(t)$ se define como:

$idf(t) = log \frac{1 + n}{1 + df(t)} + 1$

donde $n$ es el número de documentos de nuestro *corpus* y $df(t)$ es el número de documentos en los que aparece el término $t$.

Posteriormente, los vectores se normalizan a nivel de documento (el módulo del vector de cada documento es 1).

Analizando estas ecuaciones *tf-idf* observamos que aquellas palabras que tengan menor frecuencia de aparición serán más relevantes que las que aparezcan en más documentos.

Esta transformación se puede realizar utilizando el objeto [TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer).

Para ver cómo funciona, vamos a construir un *corpus* con varios documentos:

In [ ]:
corpus = [
    "Este es el primer documento...",
    "Este documento no es el primero, sino el segundo documento.",
    "Y éste es el tercer documento.",
    "¿Es éste el primero? No, es el cuarto documento."
]

Eliminamos caracteres extraños:

In [ ]:
import re
import string

# añadimos algunos más que no están en string.punctuation, como las comillas y 
# las aperturas de interrogación/exclamación
# si no los añadiésemos, no se eliminarían
chars = string.punctuation + '“”¡¿'

re_punc = re.compile('[%s]' % re.escape(chars))
# eliminar la puntuación de cada palabra
corpus = [re_punc.sub('', texto) for texto in corpus]
print(corpus)

['Este es el primer documento', 'Este documento no es el primero sino el segundo documento', 'Y éste es el tercer documento', 'Es éste el primero No es el cuarto documento']


Y acentos:

In [ ]:
letras_con_acentos = [
    'á', 'é', 'í', 'ó', 'ú'
]
letras_sin_acentos = [
    'a', 'e', 'i', 'o', 'u'
]

def quita_acentos(texto) -> str:
    res = texto
    for lca, lsa in zip(letras_con_acentos, letras_sin_acentos):
        res = res.replace(lca, lsa)
    return res

corpus = [quita_acentos(texto) for texto in corpus]
print(corpus)

['Este es el primer documento', 'Este documento no es el primero sino el segundo documento', 'Y este es el tercer documento', 'Es este el primero No es el cuarto documento']


Convertimos el texto a minúsculas:

In [ ]:
corpus = [texto.lower() for texto in corpus]
print(corpus)

['este es el primer documento', 'este documento no es el primero sino el segundo documento', 'y este es el tercer documento', 'es este el primero no es el cuarto documento']


Aplicamos la técnica Bag of Words indicándole las stop words:

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')
stop_words = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
count_vectorizer = CountVectorizer(stop_words=stop_words)
X = count_vectorizer.fit_transform(corpus)
print(X.toarray())

[[0 1 1 0 0 0 0]
 [0 2 0 1 1 1 0]
 [0 1 0 0 0 0 1]
 [1 1 0 1 0 0 0]]


In [ ]:
print(count_vectorizer.get_feature_names_out())

['cuarto' 'documento' 'primer' 'primero' 'segundo' 'sino' 'tercer']


Hasta aquí nada nuevo en el horizonte. Veamos ahora como aplicar la transformación TF-IDF:

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

In [ ]:
counts = X.toarray()
X_transformed = tfidf_transformer.fit_transform(counts)

Y analizamos el resultado:

In [ ]:
print(X_transformed.toarray())

[[0.         0.46263733 0.88654763 0.         0.         0.
  0.        ]
 [0.         0.54178991 0.         0.40927504 0.51911349 0.51911349
  0.        ]
 [0.         0.46263733 0.         0.         0.         0.
  0.88654763]
 [0.72664149 0.37919167 0.         0.5728925  0.         0.
  0.        ]]


Fijaos que ahora las ocurrencias están ponderadas de acuerdo a las ecuaciones que hemos visto antes:

$\textrm{tf-idf}(t, d) = tf(t, d) \times idf(t)$

siendo $tf(t, d)$ el número de veces que el término (palabra) $t$ aparece en el documento $d$ y $idf(t)$:

$idf(t) = log \frac{1 + n}{1 + df(t)} + 1$

donde $n$ es el número de documentos de nuestro *corpus* y $df(t)$ es el número de documentos en los que aparece el término $t$.

De esta forma, los vectores están normalizados a nivel de documento (el módulo del vector de cada documento es 1), y aquellas palabras que tengan menor frecuencia de aparición serán más relevantes que las que aparezcan en más documentos.

### Ejercicio

Realiza la limpieza del dataset, la eliminación de stop-words, la vectorización del texto (bag of words) y la TF-IDF del siguiente *corpus* de documentos:

> "Cuando se juega al Juego de Tronos, solo se puede ganar o morir." - Cersei Lannister

> "Por qué será que en cuanto un hombre construye un muro, su vecino inmediatamente quiere saber qué hay del otro lado." - Tyrion Lannister

> "¿Qué es el honor, comparado con el amor de una mujer? ¿Qué es el deber, comparado con el calor de un hijo recién nacido entre los brazos, o el recuerdo de la sonrisa de un hermano? Aire y palabras. Aire y palabras. Solo somos humanos, y los dioses nos hicieron para el amor. Es nuestra mayor gloria y nuestra peor tragedia." - Maestre Aemon, Juego de Tronos

> "El hombre que dicta la condena debe blandir la espada." - Eddard Stark

> "El poder reside donde los hombres creen que reside. Es un truco, una sombra en la pared. Y un hombre muy pequeño puede proyectar una sombra muy grande." - Lord Varys

In [ ]:
corpus = [
    "Cuando se juega al Juego de Tronos, solo se puede ganar o morir.",
    "Por qué será que en cuanto un hombre construye un muro, su vecino inmediatamente quiere saber qué hay del otro lado.",
    "¿Qué es el honor, comparado con el amor de una mujer? ¿Qué es el deber, comparado con el calor de un hijo recién nacido entre los brazos, o el recuerdo de la sonrisa de un hermano? Aire y palabras. Aire y palabras. Solo somos humanos, y los dioses nos hicieron para el amor. Es nuestra mayor gloria y nuestra peor tragedia.",
    "El hombre que dicta la condena debe blandir la espada.",
    "El poder reside donde los hombres creen que reside. Es un truco, una sombra en la pared. Y un hombre muy pequeño puede proyectar una sombra muy grande."
]